In [144]:
import pandas as pd
import requests
from io import StringIO
from math import ceil
import geopandas as gpd
import numpy as np

In [4]:
octRate = pd.read_csv("./rate_change_2021_october (1).csv")

In [157]:
octRate


,STREET_NAME,METER_CLASS,DATE_TYPE,TIME_BAND,LAST_RATE,OCCUPANCY_PCT,PROPOSED_RATE,FINAL_RATE,ADJUSTMENT_REASON
0,101-399 TERRY A FRANCOIS BLVD,Commercial,Weekday,Open-12pm,1.00,NaN,1.00,1.0,NaN
1,101-399 TERRY A FRANCOIS BLVD,Commercial,Weekday,12pm-3pm,1.00,NaN,1.00,1.0,NaN
2,101-399 TERRY A FRANCOIS BLVD,Commercial,Weekday,3pm-6pm,1.00,NaN,1.00,1.0,NaN
3,101-399 TERRY A FRANCOIS BLVD,Commercial,Weekday,6pm-Close,0.50,NaN,0.50,0.5,NaN
4,101-399 TERRY A FRANCOIS BLVD,Commercial,Weekend,Open-12pm,1.00,NaN,1.00,1.0,NaN
...,...,...,...,...,...,...,...,...,...
25743,50-99 VALLEJO ST,GMP,Weekday,6pm-Close,0.50,45.0,0.25,0.5,Meter payment realization rate threshold applied
25744,50-99 VALLEJO ST,GMP,Weekend,Open-12pm,0.50,58.0,0.25,0.5,Minimum rate rule applied
25745,50-99 VALLEJO ST,GMP,Weekend,12pm-3pm,6.25,95.0,6.50,6.5,NaN
25746,50-99 VALLEJO ST,GMP,Weekend,3pm-6pm,3.75,82.0,4.00,4.0,NaN


In [166]:
def numRangeToMiddle(str):
    rangeStr, *other = str.strip().split(" ")
    n1, n2 = rangeStr.split("-")
    mid = ceil((int(n1)+ int(n2))/2)
    return str, f"{mid} {' '.join(other)}"

In [56]:
streets = octRate["STREET_NAME"].apply(numRangeToMiddle).unique()


In [192]:
streetnames = pd.DataFrame(pd.Series(octRate["STREET_NAME"].apply(numRangeToMiddle).unique()).to_list())

In [58]:
formatted = pd.DataFrame()
formatted["0"] = list(range(1,len(streets)+1))
formatted["1"] = streets
formatted["2"] = "San Francisco"
formatted["3"] = "CA"
formatted["4"] = ""




In [75]:
streets  = formatted["1"]+formatted["2"]+formatted["3"]

In [64]:
formatted.loc[:800,].to_csv("./pre-formated.csv",index=False,header=False)


In [123]:
formatted["real-address"] = octRate["STREET_NAME"].unique()

ValueError: Length of values (1587) does not match length of index (1582)

In [78]:
from geopy.geocoders import Bing
from geopy.extra.rate_limiter import RateLimiter
geolocator = Bing(api_key="AgnofOMBhhPkWbfWoS7hZ-lqafsz5tkGlf1RLsua_XLt2FAvpZOvoHXbOPts3tdm")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0.3)

# location = geolocator.geocode("175 5th Avenue NYC")

In [81]:
locations = []
for s in streets:
    try:
        t = geolocator.geocode(s) 
    except:
        t = None
    locations.append(t)


In [83]:
s2l = pd.DataFrame()
s2l['streets']=streets
s2l["locations"] = locations

In [86]:
l0 = locations[0]

In [97]:
l0.

Point(37.77613049913857, -122.38781040811004, 0.0)

In [99]:
pt2gpt = lambda x:gpd.points_from_xy(x.latitude,x.longitude) if x else None

In [101]:
l0.longitude

-122.38781040811004

In [112]:
geom = gpd.points_from_xy([x.longitude if x else None for x in s2l.locations],
                   [x.latitude if x else None for x in s2l.locations])

In [122]:
output["real-address"] = octRate["STREET_NAME"].unique()

ValueError: Length of values (1587) does not match length of index (1582)

In [118]:
output = gpd.GeoDataFrame(s2l,geometry=geom,crs=4326)
output = output.drop("locations",axis=1)
output.to_file("./rate.geojson",driver="GeoJSON")

In [120]:
output.describe()

/Users/zh3n/opt/miniconda3/envs/spider/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,streets,geometry
count,1582,1582
unique,1582,1270
top,250 TERRY A FRANCOIS BLVDSan FranciscoCA,POINT EMPTY
freq,1,275


In [ ]:
pd.merge()

In [194]:
streetnames.columns = ["old","new"]

In [195]:
streetnames.new += "San FranciscoCA"

In [189]:
streetnames

,old,new
0,101-399 TERRY A FRANCOIS BLVD,250 TERRY A FRANCOIS BLVD FranciscoCA
1,501-599 TERRY A FRANCOIS BLVD,550 TERRY A FRANCOIS BLVD FranciscoCA
2,0-99 LILAC ST,50 LILAC ST FranciscoCA
3,2400-2499 CALIFORNIA ST,2450 CALIFORNIA ST FranciscoCA
4,300-399 08TH AVE,350 08TH AVE FranciscoCA
...,...,...
1582,600-699 TERRY A FRANCOIS BLVD,650 TERRY A FRANCOIS BLVD FranciscoCA
1583,700-799 TERRY A FRANCOIS BLVD,750 TERRY A FRANCOIS BLVD FranciscoCA
1584,800-899 TERRY A FRANCOIS BLVD,850 TERRY A FRANCOIS BLVD FranciscoCA
1585,30-98 UNION ST,64 UNION ST FranciscoCA


In [196]:
merged = output.merge(streetnames,how="left",left_on="streets",right_on="new")



In [200]:
merged2 = merged.merge(octRate,how="left",left_on="old",right_on="STREET_NAME")

In [203]:
merged2.to_file("./rate.geojson",driver="GeoJSON")